#### ***Packages***

In [ ]:
from SALib.sample import saltelli 
from SALib.analyze import sobol
import numpy as np
import pandas as pd
import os

#### ***Reading Data***

In [91]:
bp_path = "path_to_folder_stores_scaling_rate_from_binning_equal_sample_data_points"
bw_path = "path_to_folder_stores_scaling_rate_from_binning_equal_bin_width"
qr_path = "path_to_folder_stores_scaling_rate_from_quantile_regression"

In [92]:
bp_name = [file for file in os.listdir(bp_path) if file.endswith('.csv')]
bw_name = [file for file in os.listdir(bw_path) if file.endswith('.csv')]
qr_name = [file for file in os.listdir(qr_path) if file.endswith('.csv')]

In [93]:
def extract_info(filename):
    parts = filename.split('_')
    method = parts[1]
    year = parts[-1].split('.')[0]
    scenario_part = parts[-2]
    scenario = scenario_part.split('-')[-1]
    model = '-'.join(scenario_part.split('-')[:-1])
    return method, model, scenario, year

bp = []
for file in bp_name:
    df = pd.read_csv(os.path.join(bp_path, file))
    method, model, scenario, year = extract_info(file)
    df['method'] = method
    df['model'] = model
    df['scenarios'] = scenario
    df['year'] = year
    bp.append(df)
bw = []
for file in bw_name:
    df = pd.read_csv(os.path.join(bw_path, file))
    method, model, scenario, year = extract_info(file)
    df['method'] = method
    df['model'] = model
    df['scenarios'] = scenario
    df['year'] = year
    bw.append(df)
qr = []
for file in qr_name:
    df = pd.read_csv(os.path.join(qr_path, file))
    method, model, scenario, year = extract_info(file)
    df['method'] = method
    df['model'] = model
    df['scenarios'] = scenario
    df['year'] = year
    qr.append(df)

In [94]:
bp = pd.concat(bp, ignore_index = True)
bw = pd.concat(bw, ignore_index = True)
qr = pd.concat(qr, ignore_index = True)

In [95]:
df = pd.concat([bp, bw, qr], ignore_index=True)

In [96]:
df.drop(["Unnamed: 0", "slope", "intercept"], axis = 1, inplace = True)

In [97]:
df = df.pivot_table(index=['code', 'method', 'model', 'scenarios'], columns='year', values='scaling').reset_index()
df.columns.name = None
df.columns = [str(col) for col in df.columns]

#### ***Prepare for Sobol' Sensitivty Analysis***

In [100]:
gcm = df["model"].unique()
scenario = df["scenarios"].unique()
method = df["method"].unique()

In [103]:
problem = {
    'num_vars' : 3,
    'names' : ["gcm", "scenario", "method"],
    'bounds': [[0,17], [0,3], [0,3]]
}

X1 = saltelli.sample(problem, 2048*8)
X1 = X1.astype(int)

C:\Users\USER2\AppData\Local\Temp\ipykernel_29224\2405040509.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X1 = saltelli.sample(problem, 2048*8)


#### ***Prepare for Sobol' Sensitivty Analysis***

In [ ]:
uncert = {}
for code, data in df.groupby("code"):
    print(code)
    Y = np.zeros((len(X1), 31))
    for i,x in enumerate(X1):
        g,s,m = gcm[x[0]], scenario[x[1]], method[x[2]]
        d = data[(data.model == g) & (data.scenarios == s) & (data.method == m)]
        Y[i,:] = d.iloc[:,4:].values

    d = pd.DataFrame(columns=['gcm', 'scenario', 'method', 
                                      'gcm:scenario', 'gcm:method', 'scenario:method',
                                      'gcm_T', 'scenario_T', 'method_T'])
    for y in range(0, 31):
        Si = sobol.analyze(problem, Y[:,y], print_to_console=False)
        d.loc[y, 'gcm'] = Si['S1'][0]
        d.loc[y, 'scenario'] = Si['S1'][1]
        d.loc[y, 'method'] = Si['S1'][2]
        d.loc[y, 'gcm_T'] = Si['ST'][0]
        d.loc[y, 'scenario_T'] = Si['ST'][1]
        d.loc[y, 'method_T'] = Si['ST'][2]
        d.loc[y, 'gcm:scenario'] = Si['S2'][0,1]
        d.loc[y, 'gcm:method'] = Si['S2'][0,2]
        d.loc[y, 'scenario:method'] = Si['S2'][1,2]
    d['year'] = [y for y in range(2060,2091)]
    uncert[code] = d

#### ***Save as CSV files***

In [105]:
df_list = []
for code, df in uncert.items():
    df['code'] = code  
    df_list.append(df)  
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv(r"save_path\file_name.csv")